In [1]:
from pathlib import Path

import pandas as pd

from source.utils import POST_PROC_DIR, corners, timestamp_today, confirm_dir, print_md_table
from source.utils.associate import AM_DF_DIR, TOP_AM_DIR, adjust_assoc_columns
from source.utils.sample import sample_pickle as sp

K=9
DATE=timestamp_today()
FOCUS_ORIG = ['f', 'E11', 'unexpected_f',
              'am_p1_given2', 'conservative_log_ratio',
              'am_log_likelihood', 't_score',
              'mutual_information', 'am_odds_ratio_disc',
              'N', 'f1', 'f2', 'l1', 'l2']
FOCUS = adjust_assoc_columns(FOCUS_ORIG)
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_colwidth", 80)
try:
    adv_am = pd.read_csv(
    TOP_AM_DIR / f'Top{K}_NEG-ADV_combined.35f-7c_{DATE}.csv').set_index('adv')
except FileNotFoundError: 
    DATE=DATE[:-1]+str(int(DATE[-1])-1)
    adv_am = pd.read_csv(
    TOP_AM_DIR / f'Top{K}_NEG-ADV_combined.35f-7c_{DATE}.csv').set_index('adv')
# loaded_cols = adv_am.columns.to_list()
HITS_SAMPLE_PATH = TOP_AM_DIR.joinpath(f'top{K}adv_sample-{K}-hit-tables_{DATE}.pkl.gz')
adv_am

,G2_MIR,G2_SET,LRC_MIR,LRC_SET,MI_MIR,MI_SET,N_MIR,N_SET,dP1_MIR,dP1_SET,...,mean_MI,mean_N,mean_dP1,mean_expF,mean_f1,mean_f2,mean_f,mean_oddsRDisc,mean_t,mean_unexpF
adv,,,,,,,,,,,,,,,,,,,,,
exactly,"1,939.47","214,404.20",3.51,5.90,0.70,1.28,"2,032,082.00","86,330,752.00",0.59,0.67,...,0.99,"44,181,417.00",0.63,"1,231.57","1,760,088.00","31,356.50","604,556.17",1.50,110.37,"20,992.43"
necessarily,"1,688.91","219,003.46",2.66,6.23,0.60,1.30,"2,032,082.00","86,330,752.00",0.43,0.72,...,0.95,"44,181,417.00",0.57,"1,180.93","1,760,088.00","29,187.50","603,705.00",1.41,109.88,"20,658.57"
before,"1,080.52","1,062.13",5.11,3.65,0.83,1.05,"2,032,082.00","86,330,752.00",0.84,0.38,...,0.94,"44,181,417.00",0.61,35.24,"1,760,088.00",521.00,"586,969.83",1.92,15.29,265.26
that,"7,632.21","781,016.11",2.86,5.62,0.60,1.25,"2,032,082.00","86,330,752.00",0.44,0.63,...,0.93,"44,181,417.00",0.53,"5,219.08","1,760,088.00","128,932.00","657,964.83",1.32,216.58,"79,655.42"
remotely,"4,009.84","13,354.33",3.35,3.03,0.67,0.84,"2,032,082.00","86,330,752.00",0.54,0.22,...,0.75,"44,181,417.00",0.38,611.22,"1,760,088.00","12,455.50","592,102.00",1.02,49.09,"3,151.28"
yet,242.23,"209,055.78",1.18,4.74,0.39,1.14,"2,032,082.00","86,330,752.00",0.21,0.48,...,0.76,"44,181,417.00",0.34,"1,966.16","1,760,088.00","51,308.00","612,609.67",0.98,111.59,"24,466.84"
ever,"15,340.34",353.58,5.57,0.28,0.80,0.11,"2,032,082.00","86,330,752.00",0.77,0.01,...,0.45,"44,181,417.00",0.39,"2,702.62","1,760,088.00","64,885.50","610,105.33",0.95,37.38,"2,639.88"
immediately,181.20,"239,462.58",0.79,4.96,0.29,1.17,"2,032,082.00","86,330,752.00",0.14,0.52,...,0.73,"44,181,417.00",0.33,"2,032.18","1,760,088.00","52,309.50","613,753.50",0.94,116.57,"26,830.82"
any,"2,511.26","23,683.00",3.48,2.28,0.69,0.64,"2,032,082.00","86,330,752.00",0.57,0.13,...,0.67,"44,181,417.00",0.35,"1,868.76","1,760,088.00","47,833.00","605,402.67",0.94,61.22,"6,418.24"


In [2]:
adv_list = adv_am.index.to_list()
adv_list

['exactly',
 'necessarily',
 'before',
 'that',
 'remotely',
 'yet',
 'ever',
 'immediately',
 'any',
 'particularly',
 'terribly',
 'inherently',
 'altogether',
 'only']

## Load AM table for `adv~adj` comparison (bigram composition)

In [3]:
adv_adj_am = adjust_assoc_columns(
    pd.read_pickle(AM_DF_DIR.joinpath('adv_adj/RBXadj/extra/AdvAdj_frq-thrMIN-7.35f_min200x_extra.pkl.gz'))
                                  ).filter(items=FOCUS).sort_values('LRC', ascending = False)
adv_adj_am = adv_adj_am.loc[adv_adj_am.l1.isin(adv_list), :]
adv_adj_am

,f,exp_f,unexp_f,dP1,LRC,G2,t,MI,odds_r_disc,N,f1,f2,l1,l2
key,,,,,,,,,,,,,,
yet~unborn,463,0.75,462.25,0.73,10.47,"5,505.48",21.48,2.79,3.36,86330753,101707,635,yet,unborn
only~half-joking,209,1.17,207.83,0.96,9.48,"2,116.01",14.38,2.25,3.66,86330753,464174,217,only,half-joking
inherently~governmental,295,0.60,294.40,0.32,8.96,"3,178.72",17.14,2.70,2.86,86330753,55088,933,inherently,governmental
immediately~accretive,243,0.68,242.32,0.43,8.65,"2,499.70",15.55,2.56,2.80,86330753,103177,565,immediately,accretive
exactly~alike,3040,9.46,"3,030.54",0.23,8.55,"29,939.32",54.96,2.51,2.64,86330753,61599,13261,exactly,alike
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
only~high,327,"3,177.51","-2,850.51",-0.00,-2.86,"-4,245.38",-157.63,-0.99,-0.99,86330753,464174,590978,only,high
only~sure,339,"4,543.20","-4,204.20",-0.01,-3.34,"-6,708.28",-228.34,-1.13,-1.13,86330753,464174,844981,only,sure
ever~many,206,"3,193.77","-2,987.77",-0.00,-3.42,"-4,923.30",-208.17,-1.19,-1.20,86330753,124592,2212989,ever,many


In [4]:
def load_hit_table(adv_set): 
    
    # hit_df = pd.read_pickle(HITS_PATH).filter(regex=r'^[nab].*lower|text|str|head')
    # hit_df = pd.read_pickle(HITS_PATH)
    if HITS_SAMPLE_PATH.is_file(): 
        hit_df = pd.read_pickle(HITS_SAMPLE_PATH)
    else: 
        dfs = [ ]
        for i,p in enumerate(Path('/share/compling/data/sanpi/2_hit_tables/RBXadj/simple').iterdir()):
            if i  > K: 
                break
            _df = pd.read_pickle(p).filter(regex=r'^[nab].*lower|text|str|head')

            _df = _df.loc[_df.adv_form_lower.isin(adv_set), :]
            _df = _df.drop_duplicates()
            word_cols = _df.filter(regex=r'head|lower').columns
            _df[word_cols] = _df[word_cols].astype('category')
            dfs.append(_df)
        hit_df = pd.concat(dfs)
        hit_df.to_pickle(TOP_AM_DIR.joinpath(f'top{K}adv_sample-{K}-hit-tables_{DATE}.pkl.gz'))
    return  hit_df
hits_df = load_hit_table(adv_list)
hits_df.sort_index(axis=1).sample(13).iloc[:, :4]

,adj_form_lower,adv_form_lower,bigram_lower,text_window
hit_id,,,,
pcc_eng_08_090.1989_x1444001_02:13-14,fair,only,only_fair,", it 's only fair to mention that"
pcc_eng_08_053.9919_x0858120_08:8-9,wrong,terribly,terribly_wrong,"thought it went terribly wrong , but then"
pcc_eng_12_082.5650_x1317784_4:31-32,available,only,only_available,Complete Edition are only available in linen )
pcc_eng_08_033.8017_x0531124_05:11-12,available,immediately,immediately_available,% absorbed and immediately available to your body
pcc_eng_08_072.7341_x1161384_063:2-3,notable,particularly,particularly_notable,A particularly notable consequence of accidental
pcc_eng_12_077.7996_x1241025_14:21-22,worse,any,any_worse,"ca n't get any worse , right ?"
pcc_eng_08_083.6746_x1338628_12:12-13,single,ever,ever_single,"band 's first ever single , "" Dead"
pcc_eng_06_073.6434_x1175073_16:17-18,more,any,any_more,should not spend any more than a quarter
pcc_eng_06_106.9579_x1714094_8:41-42,identical,exactly,exactly_identical,"the catalysts are exactly identical . "" 3"


## Set adverb and collect specific values

In [5]:
def collect_examples(amdf, 
                     hits_df, 
                     adv: str = 'exactly', 
                     n_bigrams: int = K, 
                     n_examples: int = 50, 
                     metric: str = 'LRC') -> dict:
    df = amdf.copy().filter(like=adv, axis=0).nlargest(n_bigrams, metric)
    examples = {}
    for i, adj in enumerate(df['l2'].unique(), start=1):
        bigram = f'{adv}_{adj}'
        print(f'\n({i}) {bigram}')
        examples[bigram] = sp(
            data=hits_df, print_sample=False, 
            sample_size=n_examples, quiet=True,
            filters=[f'bigram_lower=={bigram}'],
            columns=['bigram_lower', 'text_window', 'token_str'])
        print('    > ', examples[bigram].sample(1).token_str.squeeze())
    return examples

In [6]:
for adverb in adv_am.index: 
    print(f'\n## *{adverb}*\n')
    print_md_table(adv_adj_am.filter(like=adverb, axis=0).nlargest(K, 'LRC'), n_dec=2)
    examples = collect_examples(adv_adj_am, hits_df, adv=adverb, metric='LRC')
    
    output_dir = TOP_AM_DIR / 'any_bigram_examples' / adverb
    confirm_dir(output_dir)
    print(output_dir)

    for k, df in examples.items(): 
        out_path = output_dir.joinpath(f'{k}_50ex.csv')
        print(out_path)
        df.to_csv(out_path)


## *exactly*


| key               |     f |   exp_f |   unexp_f |   dP1 |   LRC |        G2 |     t |   MI |   odds_r_disc |          N |     f1 |      f2 | l1      | l2        |
|:------------------|------:|--------:|----------:|------:|------:|----------:|------:|-----:|--------------:|-----------:|-------:|--------:|:--------|:----------|
| exactly~alike     | 3,040 |    9.46 |  3,030.54 |  0.23 |  8.55 | 29,939.32 | 54.96 | 2.51 |          2.64 | 86,330,753 | 61,599 |  13,261 | exactly | alike     |
| exactly~opposite  |   498 |    6.71 |    491.29 |  0.05 |  5.94 |  3,337.27 | 22.02 | 1.87 |          1.90 | 86,330,753 | 61,599 |   9,404 | exactly | opposite  |
| exactly~right     | 6,948 |  145.97 |  6,802.03 |  0.03 |  5.53 | 41,085.55 | 81.60 | 1.68 |          1.74 | 86,330,753 | 61,599 | 204,572 | exactly | right     |
| exactly~zero      |   344 |    8.19 |    335.81 |  0.03 |  5.02 |  1,912.07 | 18.11 | 1.62 |          1.64 | 86,330,753 | 61,599 |  11,472 | exactly | zero  